</div>
<div align="center">
  <img src="img/evolnode.png" width="800" alt="Fourier reconstruction convergence">
  <p><em> Evolution Node free you from coding and debugging, let LLM evolve your code for you.</em></p>
</div>
<div align="left">
<p><em>Bored of manual coding, a function? EvolNode let LLM automate function design and guide the evolution of it with genetic algorithm. A node here takes a task, input, and output, it uses either code or another LLM to complete the task, ensuring aligned input and output value types and ,.names.Fitness evaluation is done by running the function with a few specified test cases, the more diverse the test case, the better the evolution.
</em></p>
</div>

In [3]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode


# Code + Compilor Task
# mp = MetaPrompt("Generate Fibonacci sequence.", "fibonacci", ["n"], ["sequence"], ["int"], ["list"], PromptMode.CODE) # 
# node = EvolNode(mp, None, None)
# input_dict = {"n": 10}
# reasoning, code = node.evolve([input_dict], "i1", replace=True) # Evolution with guaranteed structural fitness
# node(input_dict) # Ok we need a output dictionary here as well ...


# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 
node = EvolNode(mp, None, None)
input_dict = {"name": "Dilireba"}
reasoning, code = node.evolve([input_dict], "i1", replace=True)
output_dict = node(input_dict) # LLM-is stochastic, so chances of mal-functioning here ... (Aim to resolve with massive inference)

In [5]:
output_dict['age']

31


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="1000" alt="Fourier reconstruction convergence">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>


In [1]:
from methods.meta_prompt import MetaPlan, extract_json_from_text

mp = MetaPlan("Help me date Dilireba, I'm a white colar worker living on salary.")
prompt = mp._get_prompt_i1()
print("MetaPrompt I1: ")
print(prompt)

from methods.llm import get_openai_response
response = get_openai_response(prompt)


MetaPrompt I1: 
Goal: Help me date Dilireba, I'm a white colar worker living on salary.
First, describe the intuition for your tactics and main steps in one sentence. The description must be inside a brace.
Generate a JSON-style plan represented as a Directed Acyclic Graph (DAG) to achieve the goal. Use creative topology in the DAG, include parallel tasks if required.

The plan should include:

- **Nodes**: Each node represents a key action or step and must contain the following attributes:
  - `task`: Description of the task.
  - `name`: Concise name used for the task function.
  - `input`: The resources, information, or prerequisites needed to perform the action.
  - `output`: The immediate result or outcome of the action.
  - `target`: The purpose or goal that the action contributes to.
  - `mode`: The execution mode for this task ("CODE" or "PROMPT").

- **Edges**: Each edge represents a dependency or relationship between nodes, indicating that one step supports or leads to another

In [2]:
from methods.meta_prompt import extract_json_from_text
from methods.diagram import visualize_plan_dict

plan_dict = extract_json_from_text(response) # used for constructing Evolution Graph
# visualize_plan_dict(plan_dict) # Visualize in d2 diagram

success: successfully compiled d2_output/plan_graph.d2 to d2_output/plan_graph.png in 242.728834ms
success: successfully compiled d2_output/plan_graph_dag.d2 to d2_output/plan_graph_dag.png in 234.71325ms
